In [ ]:
import os
import random
from time import time

import pathlib

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import skimage
from skimage import io as skimg_io
from skimage.transform import resize

import sklearn
from sklearn import  linear_model, svm, metrics
from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
%matplotlib inline

In [ ]:
IMAGE_W = 64
IMAGE_H = 64

In [ ]:
images_dir = '../input/train'
train_images_description_file_path = '../input/train.csv'

In [ ]:
train_images_description_df = pd.read_csv(train_images_description_file_path)
print(train_images_description_df.head())
print(train_images_description_df.shape)

In [ ]:
train_images_description_df_two_classes = train_images_description_df.copy()

In [ ]:
train_images_description_df_two_classes.loc[train_images_description_df_two_classes['Id'] != 'new_whale', 'Id'] = 'not_new_whale'

In [ ]:
train_images_description_df_two_classes.info()

In [ ]:
train_images_description_df_two_classes['NumId'] = 0

In [ ]:
train_images_description_df_two_classes.loc[train_images_description_df_two_classes['Id'] != 'new_whale', 'NumId'] = 1

In [ ]:
train_images_description_df_two_classes.head()

In [ ]:
for row in train_images_description_df_two_classes.head().iterrows():
    print(type(row[1]))
    print(len(row[1]))
    print(row[1])

In [ ]:
images = [
    (row[1]['Image'], row[1]['NumId'], resize(
        skimg_io.imread(
                pathlib.Path(images_dir, row[1]['Image']).as_posix(),
                as_grey=True
            ),
            (IMAGE_W, IMAGE_H)
        )
    ) for row in train_images_description_df_two_classes.iterrows()]

In [ ]:
#del images

In [ ]:
for index, image_item in enumerate(images[:6]):
    plt.subplot(2, 4, index + 1)
    plt.axis('off')
    plt.imshow(image_item[2], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Training: %s' % image_item[0])

In [ ]:
image_item = images[0]

In [ ]:
image_0 = image_item[2]

In [ ]:
type(image_0)

In [ ]:
image_0.flatten?

In [ ]:
X = np.array([image_item[2].flatten() for image_item in images])

In [ ]:
X.shape

In [ ]:
y = np.array([image_item[1] for image_item in images])

In [ ]:
y[:5]

In [ ]:
X.shape

In [ ]:
train_size = int(X.shape[0] / 5 * 4)

In [ ]:
train_size

In [ ]:
X_train = X[:train_size]
y_train = y[:train_size]
X_test = X[train_size:]
y_test = y[train_size:]

In [ ]:
svm_classifier = svm.SVC(gamma=0.001)

In [ ]:
svm_classifier.fit(X_train, y_train)

In [ ]:
y_pred = svm_classifier.predict(X_test)

In [ ]:
accuracy = metrics.accuracy_score(y_pred, y_test)

In [ ]:
print(accuracy)

In [ ]:
logr_classifier = linear_model.LogisticRegression(n_jobs=8)

In [ ]:
logr_classifier.fit(X_train, y_train)

In [ ]:
y_logr_predict = logr_classifier.predict(X_test)

In [ ]:
logr_accuracy = metrics.accuracy_score(y_logr_predict, y_test)

In [ ]:
print(logr_accuracy)

In [ ]:
#t0 = time()
'''
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              'kernel': ['linear', 'rbf']
             }
'''
param_grid = {'C': [1e3, 5e3],
              'gamma': [0.0005, 0.001],
              'kernel': ['linear', 'rbf']
             }

In [ ]:
'''
svm_multiprocess_cv_classifier = GridSearchCV(
    cross_val_score(
        svm.SVC(),
        X,
        y,
        cv=5,
        n_jobs=8
    ),
    param_grid,
    cv=5
)
'''

In [ ]:
#svm_m = clf.fit(X_train_pca, y_train)
#print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(svm_multiprocess_cv_classifier.estimator)

In [ ]:
svm_grid_search_cv_classifier = GridSearchCV(svm.SVC(), param_grid, cv=3, n_jobs=6)

In [ ]:
t0 = time()
%timeit
svm_grid_search_cv_classifier = svm_grid_search_cv_classifier.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best estimator found by grid search:")
print(svm_grid_search_cv_classifier.best_estimator_)